<a href="https://colab.research.google.com/github/vaalkate/ITMO_data_analysis/blob/main/lab5/lab_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###  Лабораторная работа №5

In [3]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import random

%matplotlib inline

In [4]:
df = pd.read_csv("beverage_r.csv", sep=";", index_col='numb.obs')
df.head()

,COKE,D_COKE,D_PEPSI,D_7UP,PEPSI,SPRITE,TAB,SEVENUP
numb.obs,,,,,,,,
1,1,0,0,0,1,1,0,1
2,1,0,0,0,1,0,0,0
3,1,0,0,0,1,0,0,0
4,0,1,0,1,0,0,1,0
5,1,0,0,0,1,0,0,0


In [5]:
len(df)

34

### Задача 1. <br>
Разделить таблицу данных так, чтобы  <br>
в обучающую выборку попало 66% наблюдений,  <br>
а в тестовую выборку 33% наблюдений

In [6]:
np.random.seed(42)

# Генерируем случайные числа для каждой строки
r = np.random.uniform(0, 1, len(df))

# Определяем обучающую выборку: 66% наблюдений
train_df = df.iloc[r <= np.quantile(r, 0.66), :]

# Остальные 33% наблюдений пойдут в тестовую выборку
test_df = df.iloc[r > np.quantile(r, 0.66), :]

print("train_df:", len(train_df))
print("test_df:", len(test_df))

print("\ntrain_df:\n", train_df)
print("\ntest_df:\n", test_df)

train_df: 22
test_df: 12

train_df:
           COKE  D_COKE  D_PEPSI  D_7UP  PEPSI  SPRITE  TAB  SEVENUP
numb.obs                                                           
1            1       0        0      0      1       1    0        1
5            1       0        0      0      1       0    0        0
6            1       0        0      0      1       1    0        0
7            0       1        1      1      0       0    1        0
11           1       0        0      0      1       1    0        0
14           1       0        0      0      0       1    0        0
15           0       1        1      0      0       0    1        0
16           0       0        0      0      1       1    0        0
17           0       1        0      0      0       1    0        0
18           1       1        0      0      1       0    0        0
19           1       0        0      0      0       0    0        1
20           1       1        1      0      1       0    0        0
22         

**Вывод по заданию 1:** таблица данных была случайным образом разделена на обучающую (66% наблюдений) и тестовую (33% наблюдений) выборки с помощью квантилей случайного распределения, при этом каждая подвыборка содержит сопоставимое количество строк.

### Задача 2. <br>
Разделить таблицу данных на четыре одинаковых части.

In [7]:
np.random.seed(42)

# Генерируем случайные числа для каждой строки
r = np.random.uniform(0, 1, len(df))

# Определяем квантили для деления на 4 части
q1, q2, q3 = np.quantile(r, [0.25, 0.5, 0.75])

# Разделяем данные на четыре примерно равные части
part1 = df.iloc[r <= q1, :]
part2 = df.iloc[(r > q1) & (r <= q2), :]
part3 = df.iloc[(r > q2) & (r <= q3), :]
part4 = df.iloc[r > q3, :]

# Выводим количество элементов в каждой части
print("Количество элементов в каждой части:")
print("Part 1:", len(part1))
print("Part 2:", len(part2))
print("Part 3:", len(part3))
print("Part 4:", len(part4))

print("\nPart 1:\n", part1)
print("\nPart 2:\n", part2)
print("\nPart 3:\n", part3)
print("\nPart 4:\n", part4)

Количество элементов в каждой части:
Part 1: 9
Part 2: 8
Part 3: 8
Part 4: 9

Part 1:
           COKE  D_COKE  D_PEPSI  D_7UP  PEPSI  SPRITE  TAB  SEVENUP
numb.obs                                                           
5            1       0        0      0      1       0    0        0
6            1       0        0      0      1       1    0        0
7            0       1        1      1      0       0    1        0
11           1       0        0      0      1       1    0        0
15           0       1        1      0      0       0    1        0
22           1       0        0      0      1       0    0        0
30           0       1        1      0      0       0    1        0
32           0       1        1      0      0       0    1        0
33           1       0        0      0      1       0    0        1

Part 2:
           COKE  D_COKE  D_PEPSI  D_7UP  PEPSI  SPRITE  TAB  SEVENUP
numb.obs                                                           
1            1     

**Вывод по заданию 2:** таблица данных была случайным образом разделена на четыре примерно равные части с помощью квантилей случайного распределения, при этом каждая подвыборка содержит сопоставимое количество наблюдений.

### Задача 3. <br>
Разделить таблицу данных на две стратифицированные подвыборки. <br>
По переменным кока-кола и минералка.

In [9]:
np.random.seed(42)

# Создаем пустые DataFrame для train и test
train_df = pd.DataFrame(columns=df.columns)
test_df = pd.DataFrame(columns=df.columns)

# Получаем уникальные комбинации COKE и TAB
for combo in df[['COKE','TAB']].drop_duplicates().itertuples(index=False):
    # Берем все строки с этой комбинацией
    subset = df[(df['COKE'] == combo.COKE) & (df['TAB'] == combo.TAB)]
    n_train = int(len(subset) * 0.66)
    # Перемешиваем подвыборку
    subset = subset.sample(frac=1, random_state=123456)
    train_df = pd.concat([train_df, subset.iloc[:n_train]])
    test_df = pd.concat([test_df, subset.iloc[n_train:]])

print("Количество элементов:")
print("Train:", len(train_df))
print("Test:", len(test_df))

print("\nРаспределение COKE и TAB в исходной выборке:\n", df[['COKE','TAB']].value_counts())
print("\nРаспределение COKE и TAB в train:\n", train_df[['COKE','TAB']].value_counts())
print("\nРаспределение COKE и TAB в test:\n", test_df[['COKE','TAB']].value_counts())

Количество элементов:
Train: 20
Test: 14

Распределение COKE и TAB в исходной выборке:
 COKE  TAB
1     0      19
0     1      10
      0       4
1     1       1
Name: count, dtype: int64

Распределение COKE и TAB в train:
 COKE  TAB
1     0      12
0     1       6
      0       2
Name: count, dtype: int64

Распределение COKE и TAB в test:
 COKE  TAB
1     0      7
0     1      4
      0      2
1     1      1
Name: count, dtype: int64


**Вывод по заданию 3:** стратифицированное разделение таблицы данных по переменным COKE и TAB позволило сохранить примерно те же пропорции комбинаций этих признаков в обучающей (66%) и тестовой (34%) выборках, что и в исходной таблице. При этом комбинации с малым числом наблюдений (например, COKE=0, TAB=1) могли попасть только в одну из выборок, что является особенностью стратификации на небольших данных.